In [92]:
import random
import pickle as pkl
import argparse
import csv
import numpy as np
import scipy.stats
import time
import pandas as pd
from sklearn.metrics import accuracy_score
from scipy.stats import chisquare

In [2]:
def load_data(ftrain, ftest):
    Xtrain, Ytrain, Xtest = [],[],[]
    with open(ftrain, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            rw = [int(x) for x in row[0].split()]
            Xtrain.append(rw)

    with open(ftest, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            rw = [int(x) for x in row[0].split()]
            Xtest.append(rw)

    ftrain_label = ftrain.split('.')[0] + '_label.csv'
    with open(ftrain_label, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            rw = int(row[0])
            Ytrain.append(rw)
    #Remove this block before submission and remove Ytest
    Ytest = []
    ftest_label = ftest.split('.')[0] + '_label.csv'
    with open(ftest_label, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            rw = int(row[0])
            Ytest.append(rw)

    print('Data Loading: done')
    return Xtrain, Ytrain, Xtest, Ytest

In [3]:
X_train, Y_train, X_test, Y_test = load_data('train.csv', 'test.csv')

Data Loading: done


In [4]:
x_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(Y_train)
x_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(Y_test)



In [5]:
y_train.columns = ['Y']


In [6]:
XY_train = pd.concat([x_train,y_train], axis = 1)

In [7]:
def infoGainOld(TD):
    #Initialize gain parameters
#     counts = TD['Y'].value_counts()
#     sysEnt = scipy.stats.entropy([x/sum(counts) for x in counts], base = 2)
    TD_length = len(TD)
#     gain = []
    min_gain = 99999
    minGainColumn = -1
    for column in TD:
        #Ignoring output column
        if column == 'Y':
            continue
        #Obtain the unique values in the column
        #Initialize column entropy to zero
        col_entropy = 0
        for value in set(TD[column]):
            #Obtain the output variable counts for one value in current column
            Y_counts = TD.loc[TD[column].isin([value])]['Y'].value_counts()
            sum_Y_counts = sum(Y_counts)
            #Calculate entropy for that value in the column
            #Add the entropy to the column entropy
            col_entropy += (sum_Y_counts/TD_length) * scipy.stats.entropy([x/sum_Y_counts for x in Y_counts], base = 2)
        #Compute column gain using system entropy
#         col_gain = sysEnt - col_entropy
#         gain.append(col_entropy)
        #Find max gain
        if col_entropy < min_gain:
            min_gain = col_entropy
            minGainColumn = column
    #Return the column having maximum gain
#     np_gain = np.array(gain[:-1])
    return minGainColumn
#     return np.argmin(np_gain)

In [99]:
# DO NOT CHANGE THIS CLASS
class TreeNode():
    def __init__(self, data='T',children=[-1]*5):
        self.nodes = list(children)
        self.data = data


    def save_tree(self,filename):
        obj = open(filename,'wb')
        pkl.dump(self,obj)

In [82]:
def ID3Algo(T, node, depth, best_attrib, p_value):
#     print(depth)
    Y_counts = T['Y'].value_counts()
    if depth == 0:
        if Y_counts[1] > Y_counts[0]:
            return TreeNode('T', [])
        else:
            return TreeNode('F', [])
#     best_attrib = infoGainOld(T)
#     if best_attrib == -1:
#         if Y_counts.index[0] == 1:
#             return TreeNode('T', [])
#         else:
#             return TreeNode('F', [])
    node.data = best_attrib
    children = []
    child_attrib = {}
    cols = set(T.columns)
    cols.remove(best_attrib)
    T_length = len(T)
    for value in set(T[best_attrib]):
        splitData = T.loc[T[best_attrib].isin([value]), cols]
        observed = splitData['Y'].value_counts()
        expected = T['Y'].value_counts() / T_length * len(splitData)
        S, p = scipy.stats.chisquare(observed,expected)
        if p > p_value:
            if len(observed) == 1:
                if observed.index[0] == 1:
                    child = TreeNode('T',[])
                else:
                    child = TreeNode('F',[])
            else:
                if observed[1] > observed[0]:
                    child = TreeNode('T', [])
                else:
                    child = TreeNode('F', [])
        else:
            if len(observed) == 1:
                if observed.index[0] == 1:
                    child = TreeNode('T',[])
                else:
                    child = TreeNode('F',[])
            else:
                best_attrib_child = infoGainOld(splitData)
                if str(best_attrib_child) in child_attrib:
                    child = child_attrib[str(best_attrib_child)]
                else:
                    child = ID3Algo(splitData, TreeNode(), depth-1, best_attrib_child, p_value)
                    child_attrib[str(best_attrib_child)] = child
        children.append((value, child))
    node.nodes = children
    return node
    

In [170]:

#Runs the ID3 algorithm to generate a decision tree
def ID3Algo(T, node, depth, best_attrib, p_value):
    Y_counts = T['Y'].value_counts()
    #Depth-limiter
    if depth == 0:
        if Y_counts[1] > Y_counts[0]:
            return TreeNode('T')
        else:
            return TreeNode('F')
    if best_attrib == -1:
            if Y_counts.index[0] == 1:
                return TreeNode('T')
            else:
                return TreeNode('F')
    node.data = best_attrib
    children = [-1,-1,-1,-1,-1]
    child_attrib = {}
    cols = set(T.columns)
    cols.remove(best_attrib)
    T_length = len(T)
    val_list = set(T[best_attrib])
    for value in range(1,6):
        #Making a guess for values not present in training set
        if value not in val_list:
            children[value - 1] = TreeNode('F')
        else:
            #Computing the p-value of each split
            splitData = T.loc[T[best_attrib].isin([value]), cols]
            observed = splitData['Y'].value_counts()
            expected = T['Y'].value_counts() / float(T_length) * float(len(splitData))
            S, p = scipy.stats.chisquare(observed,expected)

            #Comparing with the threshold
            if p > p_value:
                #pure subset case
                if len(observed) == 1:
                    if observed.index[0] == 1:
                        child = TreeNode('T')
                    else:
                        child = TreeNode('F')
                else:
                    if observed[1] > observed[0]:
                        child = TreeNode('T')
                    else:
                        child = TreeNode('F')
            else:
                #pure subset case
                if len(observed) == 1:
                    if observed.index[0] == 1:
                        child = TreeNode('T')
                    else:
                        child = TreeNode('F')
                else:
                    #Compute best attribute inside the split
                    best_attrib_child = infoGainOld(splitData)
                    #To avoid recomputation of tree at the same level, store the child node in a dict
                    if str(best_attrib_child) in child_attrib:
                        child = child_attrib[str(best_attrib_child)]
                    else:
                        child = ID3Algo(splitData, TreeNode(), depth-1, best_attrib_child, p_value)
                        child_attrib[str(best_attrib_child)] = child
            children[value - 1] = child
    node.nodes = children
    return node


In [176]:
root = TreeNode()
A= time.time()
root_tree = ID3Algo(XY_train,root, 6, 89, 1)
print(time.time() - A)

281.11544013023376


In [154]:
def DFStraversal(root):
    fringe = [root]
    leafCount = 0
    nodeCount = 0
    while len(fringe) != 0 :
        currNode = fringe.pop()
        nodeCount += 1
        if currNode.data == 'T' or currNode.data == 'F':
            leafCount += 1
        else:
            for item in currNode.nodes:
                if item != -1:
                    fringe.append(item)
    return nodeCount, leafCount

In [177]:
DFStraversal(root_tree)

(4021, 3217)

In [86]:
for value, child in root_tree.nodes:
    print(child.data)

271
167
268
80
271


In [88]:
def TreeOutputForRow(T, decisionTree):
    if decisionTree.data == 'T':
        return 1
    elif decisionTree.data == 'F':
        return 0
    else:
        for childValue, childFeature in decisionTree.nodes:
            if T[decisionTree.data] == childValue:
                return TreeOutputForRow(T, childFeature)
        return -1



In [115]:
def TreeOutput(T, decisionTree):
    outVals = []
    for idx, row in T.iterrows():
        outVals.append([TreeOutputForRow(row, decisionTree)])
    return outVals

In [123]:
# Y_predict = TreeOutput(x_test, root_tree)
tr = TreeNode('T')
tr.nodes
# print ("Accuracy is ", accuracy_score(y_test, Y_predict)*100)

[-1, -1, -1, -1, -1]

In [14]:

# observed = minnesota_table
observed = y_test[0].value_counts()
# national_ratios = national_table/len(national)  # Get population ratios

# expected = national_ratios * len(minnesota)   # Get expected counts
expected = y_train['Y'].value_counts()/len(y_train) * len(y_test)
chi_squared_stat = (((observed-expected)**2)/expected).sum()

print(chi_squared_stat)

508.1455432407473


In [15]:
scipy.stats.chisquare(observed,expected)

Power_divergenceResult(statistic=508.14554324074732, pvalue=1.6058049173024654e-112)

In [128]:
B = time.time()
T= XY_train
best_attrib = 89
chis = 0
pval = 0
cols = set(pd.DataFrame([]).columns)
cols.remove(best_attrib)
for value in set(T[best_attrib]):
    splitData = T.loc[T[best_attrib].isin([value]), cols]
#     print(splitData)
    observed = splitData['Y'].value_counts()
    print(observed)
    expected = T['Y'].value_counts() / len(T) * len(splitData)
    print(expected)
    print(scipy.stats.chisquare(observed,expected))
    S, p = scipy.stats.chisquare(observed,expected)
#     chis += S
#     pval += p
# print(time.time() - B)
print(chis)
p_value = 1 - scipy.stats.chi2.cdf(x=chis/5,  # Find the p-value
                             df=4)
print(p_value)
print(pval)

KeyError: 89

In [ ]:
crit = scipy.stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = 4)   # Df = number of variable categories - 1

print("Critical value")
print(crit)

p_value = 1 - scipy.stats.chi2.cdf(x=20,  # Find the p-value
                             df=4)
print("P value")
print(p_value)

In [45]:
infoGainOld(d)

-1

In [127]:
for i in range(274):
    print(set(XY_train[i]))

{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3, 4}
{1, 2, 3, 4, 5}
{1, 2}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4}
{1, 2}
{1, 2}
{1, 2, 3, 4}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3, 4, 5}
{1, 2, 3}
{1, 2, 3